In [23]:
# change these to try this notebook out
BUCKET = 'terrycho-training-demos-ml'
PROJECT = 'terrycho-ml'
REGION = 'us-central1'
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [24]:
%bash
grep "^def" ./babyweight/trainer/model.py

def read_dataset(prefix, batch_size=512):
def get_wide_deep():
def serving_input_fn():
def experiment_fn(output_dir):


## Check packaged code
It test package by running training with 1000 step only. <br>
"python -m {module name} " means run package with module name



In [25]:
%bash
echo "bucket=${BUCKET}"
rm -rf babyweight_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}/babyweight
python -m trainer.task \
   --bucket=${BUCKET} \
   --output_dir=babyweight_trained \
   --job-dir=./tmp \
  --pattern="00001-of-" --train_steps=1000

bucket=terrycho-training-demos-ml
gs://terrycho-training-demos-ml/babyweight/preproc/train*00001-of-*
gs://terrycho-training-demos-ml/babyweight/preproc/eval*00001-of-*


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efca19a0c10>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updat

## Run training on CloudML

Run training with CloudML and it uses distributed learning with multiple machine
In the hands on lab, plz run just 10000 steps only. 200K steps takes long time. 

In [26]:
%bash
OUTDIR=gs://${BUCKET}/babyweight/trained_model
JOBNAME=babyweight_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
#gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=$(pwd)/babyweight/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=STANDARD_1 \
   -- \
   --bucket=${BUCKET} \
   --output_dir=${OUTDIR} \
   --train_steps=2000000

gs://terrycho-training-demos-ml/babyweight/trained_model us-central1 babyweight_170721_014218
jobId: babyweight_170721_014218
state: QUEUED


Job [babyweight_170721_014218] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe babyweight_170721_014218

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs babyweight_170721_014218


In [22]:
!gcloud ml-engine jobs describe babyweight_170718_144742

createTime: '2017-07-18T14:47:45Z'
jobId: babyweight_170718_144742
startTime: '2017-07-18T14:52:46Z'
state: RUNNING
trainingInput:
  args:
  - --bucket=terrycho-training-demos-ml
  - --output_dir=gs://terrycho-training-demos-ml/babyweight/trained_model
  - --train_steps=2000000
  jobDir: gs://terrycho-training-demos-ml/babyweight/trained_model
  packageUris:
  - gs://terrycho-training-demos-ml/babyweight_170718_144742/f5e4334a00fc6c5c7d649247f9c16286b4e07ff6b92b9aead99b43d3832888e0/babyweight-0.1.tar.gz
  pythonModule: trainer.task
  region: us-central1
  scaleTier: STANDARD_1
trainingOutput:
  consumedMLUnits: 5.19

View job in the Cloud Console at:
https://console.cloud.google.com/ml/jobs/babyweight_170718_144742?project=terrycho-ml

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2Fbabyweight_170718_144742&project=terrycho-ml


Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update



## Monitoring Training Result with Tensorboard


In [27]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/babyweight/trained_model'.format(BUCKET))

TensorBoard was started successfully with pid 2943. Click here to access it.

2943

## Stop Tensorboard

In [28]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

Stopped TensorBoard with pid 2943


## Check Exported model

In [29]:
%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model/export/Servo/

gs://terrycho-training-demos-ml/babyweight/trained_model/export/Servo/
gs://terrycho-training-demos-ml/babyweight/trained_model/export/Servo/1500392990750/
gs://terrycho-training-demos-ml/babyweight/trained_model/export/Servo/1500601594617/
